In [ ]:
#註記:
    #資料內容與資料標需要做確認
    #不同帳號有不同的query_hash id(文章跟留言也不一樣)

In [ ]:
# articles        | comments        | replies
# -------------------------------------------------------               
# title        N  | id           W  |
# author_name  Y  | comment_id   Y  |
# author_id    Y  | author_name  Y  |
# context      Y  | context      Y  |
# platform_id  N  | post_id      W  |
# post_id      Y  | post_time    Y  |
# post_time    Y  | platfrom_id  N  |
# like_num     Y  | like_num     Y  |
# share_num    N  |

In [ ]:
#之後用於scrapy pipeline.py
def change_time(timeStamp):
    timeArray = time.localtime(timeStamp)
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    return otherStyleTime

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import time

headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}

In [ ]:
## 文章擷取
    #動態載入文章時，出現的網址為https://www.instagram.com/graphql/query/?query_hash=hash值&variables=向網站要求的參數
    #此為json格式，最多要求50則
    #迴圈設計:
    #    因原始api的url不需要在variables內添加"after":end_cursor，所以新設計的url需要額外添加
        #while True:
        #    resp=requests.get(url)
        #    text=json.loads(resp.text)
            #解析出end_cursor
            #解析所需要的欄位並添加進陣列
        #    if end_cursor=='null':
        #        break
        #    url=url+after:end_cursor
            
    
## 留言擷取
    
    #動態載入留言時，出現的網址為https://www.instagram.com/graphql/query/?query_hash=hash值&variables=向網站要求的參數
    #一開始請求時，可以不用加上variable 中的after(也就是end_cursor)
    #?????????或許文章也可以不用?，如果可以省掉正則表達式找取end_cursor的動作了
    
## 留言回覆擷取

In [ ]:
data_conext=[]
data_time=[]
data_like=[]
data_post_id=[]

In [ ]:
url='https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables={"id":"5513828026","first":50}'
while True:
    #all items is 50
    resp=requests.get(url,headers=headers)
    text=json.loads(resp.text)
    end_cursor=text['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']#新的end_cursor
    for i in range(0,50):
        try:
            context=text['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text']
            post_time=change_time(int(text['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['taken_at_timestamp']))
            like_num=text['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_preview_like']['count']
            post_id=text['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['id']
            print(post_id)
            print(post_time)
            print(context)
            print("按讚數:"+str(like_num))
            data_post_id.append(post_id)
            data_time.append(post_time)
            data_conext.append(context)
            data_like.append(like_num)
            print("-----------------")
        except:
            print("no item can crawl")
    
    if type(end_cursor)!=str:
        break
    time.sleep(10)
    url='https://www.instagram.com/graphql/query/?query_hash=e769aa130647d2354c40ea6a439bfc08&variables={"id":"5513828026","first":50,"after":"'+end_cursor+'"}'

In [ ]:
ddd=pd.DataFrame()
ddd['context']=data_conext
ddd['post_time']=data_time
ddd['post_id']=data_post_id
ddd['like_num']=data_like

In [ ]:
#----------目前遇到的問題---------------
#2.前12篇的留言api要求的參數也不一樣
#3.關於文章爬取的迴圈需要改善，會遇到有requests但沒有解析至陣列中
#4.文章與留言的關聯為shortcode並非我所想的post_id 所以需要跟老師討論一下式選擇其中一個還是兩個都用

In [ ]:
article_code=text['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node']['shortcode']

In [ ]:
#---------------------以下為留言表格-------------------------

In [ ]:
comment_data_comment_id=[]
comment_data_author_name=[]
comment_data_context=[]
comment_data_post_id=[]
comment_data_post_time=[]
comment_data_like_num=[]

In [ ]:
url='https://www.instagram.com/graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5&variables={%22shortcode%22:%22B9gmft5n3bT%22,%22first%22:50}'
while True:
    resp=requests.get(url,headers=headers)
    text=json.loads(resp.text)
    comment_end_cursor=text['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['end_cursor']
    i=0
    while True:
        try:
            comment_id=text['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][i]['node']['id']
            context=text['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][i]['node']['text']
            comment_time=change_time(text['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][i]['node']['created_at'])
            author_name=text['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][i]['node']['owner']['username']
            author_id=text['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][i]['node']['owner']['id']
            like_num=text['data']['shortcode_media']['edge_media_to_parent_comment']['edges'][i]['node']['edge_liked_by']['count']
            comment_data_comment_id.append(comment_id)
            comment_data_context.append(context)
            comment_data_post_time.append(comment_time)
            comment_data_author_name.append(author_name)
            comment_data_like_num.append(like_num)
            i+=1
            print(comment_id)
            print(comment_time)
            print(context)
            print(author_name)
            print(author_id)
            print(like_num)
            print('-------------------------')
        except:
            break
    if type(comment_end_cursor)!=str:
        break
    
    time.sleep(3)
    url='https://www.instagram.com/graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5&variables={%22shortcode%22:%22B9gmft5n3bT%22,%22first%22:50,"after":"'+comment_end_cursor+'"}'
    

In [ ]:
comments=pd.DataFrame()
comments['comment_id']=comment_data_comment_id
comments['comment_time']=comment_data_post_time
comments['context']=comment_data_context

In [ ]:
#---------------------以下為留言回覆表格-------------------------

In [ ]:
#註記:留言回覆若沒大於10則，應該都是能在請求留言裡面的api找出來